In [1]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
import pandas as pd
import requests

In [ ]:
# Coloca tus credenciales aquí
CLIENT_ID = ''
CLIENT_SECRET = ''
# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Coloca tus credenciales de lastfm aquí
SHARED_SECRET = ''
API_KEY = ''

last_fm_connect = requests.get(f"http://www.last.fm/api/auth/?api_key={API_KEY}")

In [5]:
paises_norte_europa = {
    "DK": "Dinamarca",
    "EE": "Estonia",
    "FI": "Finlandia",
    "IS": "Islandia",
    "IE": "Irlanda",
    "LT": "Lituania",
    "LV": "Letonia",
    "NO": "Noruega",
    "SE": "Suecia",
    "GB": "Reino Unido"}

In [6]:
def extraccion_por_paises(diccionario_zonas, años):
    todos_los_datos = [] # Esto es una lista de dicts donde se van guardando los diccionarios de cada país
    for pais in diccionario_zonas.keys():
        datos_rango = sp.search(q = f'year:{años}', limit=50, offset=0, type='track', market={pais}) # Cada una tiene que cambiar en q el rango de años
        datos_por_pais = {}
        datos_por_pais['pais'] = pais
        datos_por_pais['datos'] = datos_rango
        todos_los_datos.append(datos_por_pais)
    return todos_los_datos
   

In [7]:
def filtro_de_datos(lista_datos_zonas, datos):
    for datos_pais in lista_datos_zonas:
        for element in datos_pais['datos']['tracks']['items']:
            datos['pais'].append(datos_pais['pais'])
            datos['popularity'].append(element['popularity'])
            datos['nombre'].append(element['name'])
            artistas = []
            for artista in element['artists']:
                artistas.append(artista['name'])   
            artistas_str = ', '.join(artistas)
            datos['artista'].append(artistas_str)

In [8]:
p_norte = extraccion_por_paises(paises_norte_europa, '2015-2019')

In [9]:
dic_norte = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}

In [10]:
filtro_de_datos(p_norte, dic_norte)

In [11]:
dic_norte

{'nombre': ['All The Stars (with SZA)',
  'Skrrt',
  'Knokler Hårdt',
  'Full Automatic (feat. Diego)',
  'Hurtige Hænder - Acoustic Version - Moyo Live P3',
  'Enamorado',
  'The Night We Met',
  'Say You Love Me (Live at Ziggo Dome, Amsterdam)',
  'Pink + White',
  'Holding Back the Years (Live at Ziggo Dome, Amsterdam)',
  'White Noise Sleeping Aid to Help My Baby Fall Asleep, Sleep Through the Night',
  'Слэмятся пацаны',
  'See You Again (feat. Kali Uchis)',
  'For Your Babies (Live at Ziggo Dome, Amsterdam)',
  'Alle Skuffer Over Tid - Single Edit',
  "Greenwood, Philip Selway, Edward O'Brien, Colin Greenwood: Pyramid Song (Arr. Lawson)",
  'Lush Life',
  'So Beautiful (Live at Ziggo Dome, Amsterdam)',
  'HUMBLE.',
  'Your Mirror (Live at Ziggo Dome, Amsterdam)',
  '7 Years',
  "I'm Better (feat. Lamb)",
  'White Ferrari',
  'Someday in My Life (Live at Ziggo Dome, Amsterdam)',
  'Når Du Går Herfra (Nattely)',
  'Disciplin (feat. Keya)',
  'Perfect',
  'Wenn jetzt Sommer wär (fea

Llamada a API Last.FM y filtro para conseguir el género

In [12]:
lista_nombres_norte = list(dic_norte.values())[0]
lista_artistas_norte = list(dic_norte.values())[2]

In [13]:
def llamada_lastfm(lista_nombres, lista_artistas,datos):
    for i in range(len(lista_nombres)):
        try:
            nombre = str(lista_nombres[i]).replace(' ', '+')
            artista = lista_artistas[i].replace(' ', '+').replace(',', '+&')
            llamada_genero = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist={artista}&track={nombre}&api_key={API_KEY}&autocorrect=1&format=json")
            try:
                genero_cancion = llamada_genero.json()['toptags']['tag'][0]['name']
                datos['genero'].append(genero_cancion)
            except KeyError:
                datos['genero'].append("NULL")
                print(f"La canción {lista_nombres[i]} no está registrada en lastfm")
                
            except IndexError:
                datos['genero'].append("NULL")
                print(f"La canción {lista_nombres[i]} no tiene tags")
            
        except:
            datos['genero'].append("NULL")
            print(f"Hay problemas con la cación {lista_nombres[i]}")

In [14]:
llamada_lastfm(lista_nombres_norte, lista_artistas_norte, dic_norte)

La canción All The Stars (with SZA) no está registrada en lastfm
La canción Skrrt no está registrada en lastfm
La canción Full Automatic (feat. Diego) no está registrada en lastfm
La canción Hurtige Hænder - Acoustic Version - Moyo Live P3 no tiene tags
La canción Say You Love Me (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción Pink + White no tiene tags
La canción Holding Back the Years (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción Слэмятся пацаны no está registrada en lastfm
La canción See You Again (feat. Kali Uchis) no está registrada en lastfm
La canción For Your Babies (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción Alle Skuffer Over Tid - Single Edit no tiene tags
La canción Greenwood, Philip Selway, Edward O'Brien, Colin Greenwood: Pyramid Song (Arr. Lawson) no está registrada en lastfm
La canción So Beautiful (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción Your Mirror (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción I'm Better (feat

In [15]:
df_norte = pd.DataFrame(dic_norte)

In [16]:
df_norte.index.name="id"

In [17]:
df_norte

,nombre,popularity,artista,pais,genero
id,,,,,
0,All The Stars (with SZA),90,"Kendrick Lamar, SZA",DK,NULL
1,Skrrt,7,"K27, Einar",DK,NULL
2,Knokler Hårdt,50,Gilli,DK,electronic
3,Full Automatic (feat. Diego),0,"Sleiman, Diego",DK,NULL
4,Hurtige Hænder - Acoustic Version - Moyo Live P3,50,The Minds Of 99,DK,NULL
...,...,...,...,...,...
495,Sirenen,0,Tommy,GB,NULL
496,Tennessee Whiskey,78,Chris Stapleton,GB,country
497,Did You Love Me,0,Shannon & the Clams,GB,NULL


In [18]:
df_norte.to_csv("paises_norte_2015_2019.csv")